
# <p style="text-align: center;">EE 380L: Data Mining</p>
# <p style="text-align: center;">Assignment 3</p>
## <p style="text-align: center;">Total points: 80</p>
## <p style="text-align: center;">Due: Tuesday, March 20th, submitted via Canvas by 11:59 pm</p>

Your homework should be written in a **Jupyter notebook**. You may work in groups of two if you wish. Only one student per team needs to submit the assignment on Canvas.  But be sure to include name and UTID for both students.  Homework groups will be created and managed through Canvas, so please do not arbitrarily change your homework group.  

Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting. (%matplotlib inline)

# Question 1 - Gradient Descent (5+12+3 = 20pts)

In this question you will implement vanilla SGD and an adaptive gradient update technique called Adagrad. In addition, you will also implement ridge regularization. 

1. Using stochastic gradient descent, derive the coefficent updates for all 4 coefficients of the model: $$ y = w_0 + w_1*x_1 + w_2*x_1*x_1 + w_3*x_2*x_2 + w_4*x_2 $$ Hint: start from the cost function (Assume sum of squared error). If you write the math by hand, submit that as a separate file and make a reference to it in your notebook or include the image in your notebook.
2. Write Python code for an SGD solution to the non-linear model $$ y = w_0 + w_1*x_1 + w_2*x_1*x_1 + w_3*x_2*x_2 + w_4*x_2 $$ Try to format similarly to scikit-learn's models. Your Python class should take as input the learning_rate, regularization_constant and number of epochs. The fit method must take as input X,y and a choice of update_rule as 'SGD', 'adagrad' or 'adadelta' (Notes on implementation below). The _predict_ method takes an X value (optionally, an array of values). Use your new gradient descent regression to predict the data given in 'samples.csv', for 20 epochs (this may have to increase for adadelta), using learning rates: [.0001, .001, .01, 0.1, 1, 10, 100] and regularization constants in the range: [0,10,100] . Plot MSE and the $w$ parameters as a function of epoch count (20 epochs) for the best combination of learning_rate and regularization for both SGD, Adagrad and Adadelta. ie you should have a plot of MSE and parameter updates for SGD, adagrad and adadelta. Report the MSE at the end of 10 epochs for all 3.
3. Based on the experiments, answer the following:

1. Among SGD and Adagrad, which one allows for larger initial setting of the learning_rate? Why?
2. Mention one benefit and one drawback of Adadelta over Adagrad

# Notes on Adagrad/Adadelta
Adagrad (http://ruder.io/optimizing-gradient-descent/) differs from vanilla SGD in that the learning rate of each weight changes over updates. A cache is maintained that holds the sum of squares of all gradients upto the current update. The learning_rate is divided by the cache, resulting in a different learning rate for each weight. A consequence of this update rule is that weights that have already seen large gradients (made large jumps) make smaller updates in subsequent iterations.
Specifically, the steps can be listed as below:
1. cache = cache + (gradients^2)
2. weights = weights + ((learning_rate)/sqrt(cache+1e-6))*gradients

The key difference between Adadelta and Adagrad is that the former uses a weighted sum for each update of the cache. Also, you will not need to use a learning rate to make Adadelta work. In addition to the link above, the paper on Adadelta is a great resource to implement it correctly (https://arxiv.org/pdf/1212.5701.pdf)

# (Optional) Notes on Training with Gradient Descent
1. Compute error: This consists of a prediction error and a regularization term. From an implementation perspective, this is a function that takes as input the truth, prediction and regularization hyperparameter and returns an error
2. Compute gradients: Take a derivative of the error in terms of the weights. This can be modelled as a function that takes as input the error and features and returns the gradients for each weight
3. Update weights: Weight updates can be done using vanilla SGD or adaptive techniques. The update function takes as inputs the gradient and hyperparameters and returns the new weights.

# 2. (6+4=10 pts) Tensor Playground
Visit http://playground.tensorflow.org for this problem

A. From the far right, select "Regression" as the problem type, and select the 2nd of the two data sets ( the right one ).  

   i) What sort of test / training loss do you get if you run it for 200 epochs with the following learning rates: .3, .01 and .003 ?  What if you run it for 1000 epochs with these learning rates?  Leave all other values at their defaults ( test/training ratio 50%, Noise 0, Batch Size 10, using Tanh activation function, and No Regularization )
   
   ii) Keeping learning rate at .3, Activation at Tanh, with all others at their defaults, and running for 200 epochs.  
     What sort of test/train loss can you achieve using only 1 neuron in the first hidden layer.  What about for 2,3 or 8 neurons?  Provide screen shots of output layer and comment on how the different output fits look and compare with one another.

   iii)Now keeping learning rate at .03 with all others at their defaults, and running for 200 epochs.  
       Compare how the activation functions affect the ouput ( ReLU, Sigmoid, Tanh, Linear ). Provide screen shots of output results and comment.


B. Neural Nets can fit anything.  Now reset to the initial defaults, and select "Classification" as the problem type, and from the Data section, select the bottom right "Spriral" data set.  With the idea of trying to minimize training/testing error, provide solutions to the problem for the following 2 scenarios.  i) Using just the first 2 inputs ( as per default ) and ii) Using all 7 of the inputs.  You may use as many layers as you want, whatever activation, however man neurons.  Provide screen shots which show your full network, output and parameters. Briefly justify your decisions, and comment on difficulties/tradeoffs, what helps/what doesn't,etc. 

## ANSWER

# Question 3: Multi-layer Perceptron Regressor (15 points)

In this question, you will explore the application of Multi-layer Perceptron (MLP) regression using sklearn package in Python. We will use the OpenCL gemm kernel performance prediction dataset for this problem https://archive.ics.uci.edu/ml/datasets/SGEMM+GPU+kernel+performance.

Following code will pre-process the data and split the data into training and test set using [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) with **random state 42** and **test_size = 0.33**.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,KFold)
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import glob
%matplotlib inline

data = pd.read_csv('sgemm_product.csv') 
data['target'] = 0.25*(data['Run1 (ms)'] + data['Run2 (ms)']+ data['Run3 (ms)'] + data['Run4 (ms)'])
y = data['target']
X = data.drop(['target','Run1 (ms)', 'Run2 (ms)', 'Run3 (ms)', 'Run4 (ms)'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33, random_state=42)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

((161872, 14), (161872,), (79728, 14), (79728,))


One more thing to use in this problem is [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). Instead of fitting a model on original data, use StandardScaler to make each feature centered ([Example](http://scikit-learn.org/stable/auto_examples/applications/plot_prediction_latency.html#sphx-glr-auto-examples-applications-plot-prediction-latency-py)). Whenever you have training and test data, fit a scaler on training data and use this scaler on test data. Here, scale only features (independent variables), not target variable y.

Use [sklearn.neural_nework.MLPRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor) to do a 5-fold cross validation using sklearn's [KFold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold). The cross validation must be performed on the **training data**. 

1) Use following parameter settings for MLPRegressor:

    activation = 'tanh', solver = 'sgd', learning_rate='constant', random_state=42,
    batch_size=5000, learning_rate_init = 0.005
    
Now, consider the following settings for the number of hidden units:
    
        (2,), (10,), (20,), (30,), (40,)
    
   Report the average Root Mean Squared Error (RMSE) value based on your 5-fold cross validation for each model (5pts)
      
2) Now, using the same number of hidden units used in part 1), train MLPRegressor models on whole training data and report RMSE score for both Train and Test set (Again, use StandardScaler). Which model works the best? Briefly analyze the result in terms of the number of hidden units. (3pts)

3) MLPRegressor has a built-in attribute *loss\_curve\_* which returns the loss at each epoch (misleadingly referred to as "iteration" in scikit documentation, though they use epoch in the actual code!). For example, if your model is named as *my_model* you can call it as *my\_model.loss\_curve\_* ([example](http://scikit-learn.org/stable/auto_examples/neural_networks/plot_mlp_training_curves.html#sphx-glr-auto-examples-neural-networks-plot-mlp-training-curves-py)). Plot three curves for model (a), (b), and (c) in one figure, where *X-axis* is epoch  number and *Y-axis* is squared root of *loss\_curve\_* value. (2pts)

4) Activation Layer: Use the tanh and relu activations for the following fixed parameters to train your model:
    
    solver = 'sgd', random_state=42,
    batch_size=5000, hidden_layer_sizes = (2,)

   Report the average Root Mean Squared Error (RMSE) value based on your 5-fold cross validation for each model. (2pts)
   
5) Optimization technique: The sklearn API performs a momentum update when using SGD. To understand the difference in performance of vanilla SGD and SGD with momentum, set momentum to 0 with SGD. Here, we compare the performance of vanilla sgd against adam solvers. Since both of these use different optimization subroutines, the learning rate must also be varied. In this question, vary the learning rate through these values [1, 0.1, 0.01, 0.001]. Keep the following fixed parameters to train your model:

    activation = 'tanh', random_state=42,
    batch_size=5000, hidden_layer_sizes = (2,), momentum = 0
    
   Report the average Root Mean Squared Error (RMSE) value based on your 5-fold cross validation for the best learning rate for each model. (3pts)

## Solution

### 1)

# Question 4: Multi-level Model in Python (19 points)
In this problem, you will explore multi-level model using a python package [PyMC3](https://pymc-devs.github.io/pymc3/index.html#). The dataset "brew_by_year.csv" will be used in this question.  This dataset contains fields:

- `state`
- `state_id`
- `brew`
- `year`

Where `brew` is the number of breweries in the state that year, and `state_id` is a unique ID for that state.

MLM is a Bayesian modeling technique which assumes a prior distribution over the parameters. This allows us to define (multilevel) relationships between the parameters. In general, this yields a posterior that is, in general, intractable. However, we can still perform inference using Monte-Carlo Markov Chain methods by generating samples from this distribution! PyMC3 helps us do this. Also, remember that this is a Bayesian method; we are estimating a distribution for each parameter rather than just a single value.


(a)  (2 pts) Plot the relationship between number of breweries and year, and draw a linearly regressed line ignoring the ID variable. Use `LinearRegression` from `sklearn`.

(b)  (2 pts) Plot the relationship between number of breweries and year, but this time, fit a different linear regression for each state. Use `LinearRegression` from `sklearn`.

(c)  (2 + 4 + 6 pts) Divide the dataset into training and test sets.  The training set contains the first 6 years of the measurements, and the test set contains the rest of the measurements. 
Build three different linear models:
- Global model:  a linear model using `brew` as the dependent variable and `year` as the independent variable. Pool all data and estimate one common regression model to assess the influence of the passage of time across the total number of breweries. Use `LinearRegression` from `sklearn`. This is similar to part a).
- Local model:  a different linear model for each state i.e., 51 different linear regressions. We are interested in whether different states actually follow separate regression models. Use `LinearRegression` from `sklearn`. This is similar to part b).
- Multilevel model:  Use the PyMC3 package to fit a multilevel model specified as follows:

\begin{align*}
\text{brew}_{it} &= \beta_{0i} + \beta_{1i} \text{year}_{it} + \epsilon_{it}\\
\beta_{0i} &= \beta_{00}  + \eta_{0i} \\
\beta_{1i} &= \beta_{10} + \eta_{1i} \\
\end{align*}

Predict the number of breweries for the remaining years, and calculate the mean squared errors from the three models. You may find this link helpful: [Partial pooling: Hierarchical Regression aka, the best of both worlds](http://pymc-devs.github.io/pymc3/notebooks/GLM-hierarchical.html#Partial-pooling:-Hierarchical-Regression-aka,-the-best-of-both-worlds). Note the usage of `pm.sample` which generates samples from the posterior and `pm.traceplot` which provides a nice set of plots showing the estimated posterior distribution. We assume a gaussian prior over the coefficients of our linear model, so the most likely estimate is the mean of the samples from the distribution. To calculate the MSE of the MLM, take the average over the MSEs at the state level.

(d)  (2 pts) Briefly state what do $\beta_{00}$ and $\beta_{10}$ mean in this multilevel model.

(e) (1 pt) Visually assess from the trace plot whether the MLM specified in the problem is a reasonable choice.

In [ ]:
import pymc3 as pm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
import numpy as np

data = pd.read_csv('brew_by_year.csv', index_col=0)
data['brew'] = (data['brew'] - np.mean(data['brew'])) / np.std(data['brew'])

# Specify the hyper-parameter of Multilevel model:
beta_00 = pm.Normal('mu_beta0', mu=0., sd=10)
eta_0i = pm.HalfCauchy('eta_0', 5)
beta_10 = pm.Normal('mu_beta1', mu=0., sd=10)
eta_1i = pm.HalfCauchy('eta1', 5)
    
# Intercept and Slope
beta_0i = pm.Normal('beta0', mu=beta_00, sd=eta_0i, shape=len(train_data.state_id.unique()))
beta_1i = pm.Normal('beta1', mu=beta_10, sd=eta_1i, shape=len(train_data.state_id.unique()))    
# Model error
eps = pm.HalfCauchy('eps', 5)

## Answer

# Question 5: Decision Tree using Python (5 pts)
In this problem, you will model the data using decision trees to perform a classification task on the energy dataset provided. The dataset has been preprocessed. Using the class tree.DecisionTreeClassifier (http://scikit-learn.org/stable/modules/classes.html#module-sklearn.tree), build two different trees with a maximum depth of two using the split criteria (i) Gini and (ii) Entropy. Use entire data to build trees. Plot the two trees. If your classifier object is called clf, use the following commands to save the generated tree as a '.dot' file that can be used to visualize the tree using Webgraphviz: http://www.webgraphviz.com/

Hint: see  http://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html

Copy paste the contents of your '.dot' file into the text window on the website to visualize the trees. At which node(s) do they differ?

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('energydata_complete.csv') 

y = data['Appliances']
X = data.drop(['date','Appliances', 'rv1', 'rv2'], axis = 1)


from dateutil import parser
timeData = np.array(data['date'])

days = []
hours = []
for line in xrange(len(timeData)):
    day = parser.parse(timeData[line]).weekday()
    hour = parser.parse(timeData[line]).hour
    days.append(day)
    hours.append(hour)
    
X = pd.concat([X, pd.get_dummies(days), pd.get_dummies(hours)], axis = 1)

# 6. (2+5+3=10 pts) Bayes Decision Theory
a. Explain what you understand by class-conditional likelihood, class priors, and posterior probability of a class given an input, and the relationship between them. Please define all symbols and equations used explicitly.
( 2 points )

b. Suppose you want to learn a binary classifier to predict whether or not a customer will buy a TV. The class label is 0 if the customer does not buy a TV and 1 if they do. For each customer, you are given two features, $x_1$ is the per hour salary and $x_2$ is the age. Assume that the class conditional distribution $p([x_1 , x_2]|C)$ is Gaussian. The mean salary and age of the people who do buy a TV is 15 and 40 respectively and that of those who don’t is 8.5 and 25. The first class of customers, $\sigma_1$ = 1, $\sigma_2 = 2$ and $\rho = 0$. For the second class of customers, $\sigma_1$ = 3, $\sigma_2 = 2$ and $\rho = 0$.(Refer HW1 on how to construct a covariance matrix with this information). Further, your sales data suggests that only 1 in 3 people actually bought a TV in the last few years. Mathematically derive the (optimal) Bayes decision boundary for this problem. (5 points)

c. Now sample 1000 customers from each class (C = 0, 1) under the assumed distribution and the estimated parameters and plot their features. Additionally, plot the decision boundary you obtained in the part (b) on the same plot. (3 points)

## ANSWER